In [1]:
import copy 
import random
import numpy as np

In [167]:
class Board:

    def __init__(self, code=None):
        self.resetBoard()
        self.board = None 
        if code:
            self.fill_code(code)
        else:
            self.code = None
            
    def fill_code(self,code):
            self.code = code
            for row in range(9):
                for col in range(9):
                    self.board[row][col] = int(code[0])
                    code = code[1:]

    def boardToCode(self, input_board=None):
        if input_board:
            _code = ''.join([str(i) for j in input_board for i in j])
            return _code
        else:
            self.code = ''.join([str(i) for j in self.board for i in j])
            return self.code


    def findSpaces(self): 
        #row = [(row, col) for col in range(len(self.board[0])) for row in range(len(self.board)) if self.board[row][col] == 0]
        #return row
        for row in range(len(self.board)):
            for col in range(len(self.board[0])):
                if self.board[row][col] == 0:
                    return (row, col)
        return False


    def checkSpace(self, num, space):
        if not self.board[space[0]][space[1]] == 0: 
            return False

        for col in self.board[space[0]]:
            if col == num:
                return False

        for row in range(len(self.board)):
            if self.board[row][space[1]] == num:
                return False

        internalBoxRow = space[0] // 3
        internalBoxCol = space[1] // 3

        for i in range(3):
            for j in range(3):
                if self.board[i + (internalBoxRow * 3)][j + (internalBoxCol * 3)] == num:
                    return False

        return True


    def solve(self): 
        _spacesAvailable = self.findSpaces()

        if not _spacesAvailable:
            return True
        else:
            row, col = _spacesAvailable

        for n in range(1, 10):
            if self.checkSpace(n, (row, col)):
                self.board[row][col] = n

                if self.solve():
                    return self.board

                self.board[row][col] = 0

        return False



    def findNumberOfSolutions(self):
            _z = 0
            _list_of_solutions = []

            for row in range(len(self.board)):
                for col in range(len(self.board[row])):
                    if self.board[row][col] == 0:
                        _z += 1

            for i in range(1, _z+1):
                _board_copy = copy.deepcopy(self)

                _row, _col = self.__findSpacesToFindNumberOfSolutions(_board_copy.board, i)
                _board_copy_solution = _board_copy.__solveToFindNumberOfSolutions(_row, _col)

                _list_of_solutions.append(self.boardToCode(input_board=_board_copy_solution))

            return list(set(_list_of_solutions))


    def findSpacesToFindNumberOfSolutions(self, board, h):
        _k = 1
        for row in range(len(board)):
            for col in range(len(board[row])):
                if board[row][col] == 0:
                    if _k == h:
                        return (row, col)
                    _k += 1
        return False


    def solveToFindNumberOfSolutions(self, row, col): 
        for n in range(1, 10):
            if self.checkSpace(n, (row, col)):
                self.board[row][col] = n

                if self.solve():
                    return self.board

                self.board[row][col] = 0

        return False


    def resetBoard(self):
        self.board = [[0 for _ in range(9)] for _ in range(9)]
        return self.board





In [168]:
class Generator(Board):
    def __init__(self, difficulty):
        self.difficulty = difficulty
    
    def generateQuestionBoardCode(self): 
        self.board, _solution_board = self.generateQuestionBoard(self.generateRandomCompleteBoard())
        return self.boardToCode(), self.boardToCode(_solution_board)
    
    def generateQuestionBoard(self, fullBoard):
        self.board = copy.deepcopy(fullBoard)

        if self.difficulty == 0:
            _squares_to_remove = 36
        elif self.difficulty == 1:
            _squares_to_remove = 46
        elif self.difficulty == 2:
            _squares_to_remove = 52
        else:
            return

        _counter = 0
        while _counter < 4:
            _rRow = random.randint(0, 2)
            _rCol = random.randint(0, 2)
            if self.board[_rRow][_rCol] != 0:
                self.board[_rRow][_rCol] = 0
                _counter += 1

        _counter = 0
        while _counter < 4:
            _rRow = random.randint(3, 5)
            _rCol = random.randint(3, 5)
            if self.board[_rRow][_rCol] != 0:
                self.board[_rRow][_rCol] = 0
                _counter += 1

        _counter = 0
        while _counter < 4:
            _rRow = random.randint(6, 8)
            _rCol = random.randint(6, 8)
            if self.board[_rRow][_rCol] != 0:
                self.board[_rRow][_rCol] = 0
                _counter += 1

        _squares_to_remove -= 12
        _counter = 0
        while _counter < _squares_to_remove:
            _row = random.randint(0, 8)
            _col = random.randint(0, 8)

            if self.board[_row][_col] != 0:
                n = self.board[_row][_col]
                self.board[_row][_col] = 0

                if len(self.findNumberOfSolutions()) != 1:
                    self.board[_row][_col] = n
                    continue

                _counter += 1

        return self.board, fullBoard

    def generateRandomCompleteBoard(self):
            self.resetBoard()
            _l = list(range(1, 10))
            for row in range(3):
                for col in range(3):
                    _num = random.choice(_l)
                    self.board[row][col] = _num
                    _l.remove(_num)

            _l = list(range(1, 10))
            for row in range(3, 6):
                for col in range(3, 6):
                    _num = random.choice(_l)
                    self.board[row][col] = _num
                    _l.remove(_num)

            _l = list(range(1, 10))
            for row in range(6, 9):
                for col in range(6, 9):
                    _num = random.choice(_l)
                    self.board[row][col] = _num
                    _l.remove(_num)

            return self.generateCont()
        
    def findNumberOfSolutions(self):
        _z = 0
        _list_of_solutions = []

        for row in range(len(self.board)):
            for col in range(len(self.board[row])):
                if self.board[row][col] == 0:
                    _z += 1

        for i in range(1, _z+1):
            _board_copy = copy.deepcopy(self)

            _row, _col = self.findSpacesToFindNumberOfSolutions(_board_copy.board, i)
            _board_copy_solution = _board_copy.solveToFindNumberOfSolutions(_row, _col)

            _list_of_solutions.append(self.boardToCode(input_board=_board_copy_solution))

        return list(set(_list_of_solutions))
    
    def generateCont(self): 
        for row in range(len(self.board)):
            for col in range(len(self.board[row])):
                if self.board[row][col] == 0:
                    num = random.randint(1, 9)

                    if self.checkSpace(num, (row, col)):
                        self.board[row][col] = num

                        if self.solve():
                            self.generateCont()
                            return self.board

                        self.board[row][col] = 0

        return False

In [169]:
class Solver(Board):
    def __init__(self,code,board):
        self.code = code
        self.board = board
    
    def solveForCode(self):
        return self.boardToCode(self.solve())
    
    def solve(self): 
        _spacesAvailable = self.findSpaces()

        if not _spacesAvailable:
            return True
        else:
            row, col = _spacesAvailable

        for n in range(1, 10):
            if self.checkSpace(n, (row, col)):
                self.board[row][col] = n

                if self.solve():
                    return self.board

                self.board[row][col] = 0

        return False

        
    

In [174]:
gen = Generator(0) # get 0 for easy level, 1 for medium and 2 for hard
question_board_code = gen.generateQuestionBoardCode()
code = question_board_code[0]
solution = question_board_code[1]
solved_board_code = Solver(code,gen.board).solveForCode()

In [175]:
code

'080051304001640590060308017140000006029063850608000409075010042492736005806024900'

In [176]:
solved_board_code

'987251364231647598564398217143985726729463851658172439375819642492736185816524973'

In [177]:
solution

'987251364231647598564398217143985726729463851658172439375819642492736185816524973'